# Info bici

## Piste ciclabili

In [ ]:
import json

In [ ]:
PATH_CICLABILI_RAW = "./Data/Raw/Info_bici/bike_ciclabili_raw.geojson"
PATH_CICLABILI_CLEAN = "./Data/Clean/Info_bici/bike_ciclabili_clean.geojson"

### ETL (solo conversione di formato)

Carichiamo il file Raw geojson

In [ ]:
with open(PATH_CICLABILI_RAW, 'r') as file:
    geojson = json.load(file)

Nessuna modifica necessaria al momento

Salviamo il file modificato geojson in Clean

In [ ]:
with open(PATH_CICLABILI_CLEAN, 'w') as file:
    json.dump(geojson, file, indent=4)

## Stazioni BikeMi

In [ ]:
import json

import geopandas as gpd

In [ ]:
PATH_BIKEMI_RAW = './Data/Raw/Info_bici/bikemi_stazioni_raw.csv'
PATH_BIKEMI_CLEAN = "./Data/Clean/Info_bici/bikemi_stazioni_clean.geojson"

### ETL

Leggiamo il file csv Raw

In [ ]:
gdf = gpd.read_file(PATH_BIKEMI_RAW, sep = ';')

Rinominiamo alcune colonne

In [ ]:
gdf = gdf.rename(columns = {'id_amat':'id_stazione',
                           'stalli':'n_posti',
                           'LONG_X_4326':'longitudine',
                           'LAT_Y_4326': 'latitudine',
                           'Location':'location'})

Eliminiamo le colonne che non ci servono

In [ ]:
gdf.drop(['stato', 'id_via', 'indirizzo', 'civico', 'zd_attuale', 'anno', "tipo"], axis=1, inplace=True)

Aggiungiamo una colonna per l'icona da visualizzare su kepler

In [ ]:
gdf['icon'] = "events"

Convertiamo il csv in geojson

In [31]:
geojson = {
    "type": "FeatureCollection",
    "name": "",
    "crs": {
        "type": "name",
        "properties": {
            "name": "urn:ogc:def:crs:EPSG::6707"
        }
    },
    "features": []
}
geojson["name"] = "BikeMi_Stazioni"
for i in gdf.index:
    feature = {
        "type": "Feature",
        "properties": {},
        "geometry": {
            "type": "",
            "coordinates": []
            }
        }
    for j in gdf.columns:
        if j == "location":
            feature["geometry"]["type"] = "Point"
            coordinates = gdf.at[i, j].strip("(").strip(")").strip().split(",")
            feature["geometry"]["coordinates"].append(float(coordinates[1]))
            feature["geometry"]["coordinates"].append(float(coordinates[0]))
        else:
            feature["properties"][j] = gdf.at[i, j]
    geojson["features"].append(feature)

Salviamo il geojson nei Clean

In [ ]:
with open(PATH_BIKEMI_CLEAN, 'w') as file:
    json.dump(geojson, file, indent=4)